\# Copyright 2024 Loic Domaigne <br>
\# SPDX-License-Identifier: Apache-2.0 

# Solving the ITWM PI- day challenge

The [ITWM](https://www.itwm.fraunhofer.de/en.html) submitted a [math puzzle on LinkedIn](https://www.linkedin.com/posts/fraunhofer-itwm_idm2024-iday24-activity-7174061183384047617) for the International Mathematics Day (IDM) on March 14th - aka PI-day:

> insert the missing symbols, to solve the equation:
> 
> $ 1 \quad 7 \quad  6 \quad  2 \quad 4 \quad = \quad 100$
>
>Fill the missing symbols to get the equation right! Whether plus, minus, times, divided or brackets, everything is possible.

## 1. Solutions found by participants 

$
\begin{flalign*}
(17+6+2)* 4 = 100 \\
(1+7)*6*2+4 = 100 \\
17*6 + 2 - 4 = 100
\end{flalign*}
$

So, we know that solutions to the problem exist. But did we find all of them? Let's see if we can figure it out with Python and some human Intelligence.

## 2. First idea (not good enough)

A first idea is to consider the left of side of the equation as an expression using different operations $S_0, S_1,...$:

$ 1 \quad  S_0 \quad 7 \quad S_1 \quad 6 \quad S_2 \quad 2 \quad S_3 \quad 4$

where $ S_k $ is one of the $ \{+,-,*,/,\circ \} $ symbol. The $\circ$ symbol represents the "empty symbol", leading to string concatenation. For instance, $1 \circ 7 = 17$.

A simple brute force algorithm consists to form the expressions corresponding to all possible symbols combination, and evaluate if the value is 100. Of course, this approach suffers from combinatorial explosion, but the size of the search space here is small: $5^4 = 625$ if we assume one symbol between each digit.

Let's put this idea into code. First, we can generate all symbol combinations using [product()](https://docs.python.org/3/library/itertools.html#itertools.product) from [itertools](https://docs.python.org/3/library/itertools.html#module-itertools). This function returns a itertor, which will produce one of the combination at each iteration.

In [1]:
from itertools import product

symbols=('+','-','*','/','')
it=product(symbols, repeat=4)
next(it)

('+', '+', '+', '+')

In [2]:
S = next(it)
print(S)

('+', '+', '+', '-')


Given one combination S, we can form the corresponding expression using a f-string<br/>
`expr = f"1{S[0]}7{S[1]}6{S[2]}2{S[3]}4"`

Let's print out for a few iterations:

In [3]:
it=product(symbols, repeat=4)
for n,S in enumerate(it):
    expr = f"1{S[0]}7{S[1]}6{S[2]}2{S[3]}4"
    print(expr)
    if n==5:
        break

1+7+6+2+4
1+7+6+2-4
1+7+6+2*4
1+7+6+2/4
1+7+6+24
1+7+6-2+4


Looks good! We just have now to evaluate the expressions using [eval()](https://docs.python.org/3/library/functions.html#eval), and check if the value is 100...

In [4]:
it=product(symbols, repeat=4)
for n,S in enumerate(it):
    expr = f"1{S[0]}7{S[1]}6{S[2]}2{S[3]}4"
    if eval(expr)==100:
        print(expr)
print(f"Explored {n+1} solutions")

1*76+24
17*6+2-4
Explored 625 solutions


Fabulous! We have found a solution that no one had found: 
> $1*76+24$. 

Cool 😎 

## 3. Missing solutions

But we're obviously missing some solutions, like $(17+6+2)* 4 = 100$. <br/>
What's missing. Oh dear? the braces...

Can we extend our existing implementation to include the brace symbols `(` and `)`?  If we want to find the previous solution missed, we need to looks for expression of the kind:

$
\begin{matrix} 
( & 1 &     & 7 &  +  & 6 &  +  & 2 &  )  & *   & 4 \\
S_0 & 1 & S_1 & 7 & S_2 & 6 & S_3 & 2 & S_4 & S_5 & 4
\end{matrix}
$

First, the space has growth from $5^4 = 625$ to $7^6=117649$. That's manageable, but we're still aren't exploring all the possible solutions. How about:

$
\begin{align*}
(1+7)*(6+2)+4 \\
((1+7)*6-2)/4
\end{align*}
$

Beside the search space exploding in size, the second difficulty is to form the expressions we want to investigate. Taking the previous examples:

$
\begin{matrix}
  ( & 1 &  +  & 7 &  )  &  *  &  (  & 6 &  +  & 2 &  )  &  +  & 4 \\
S_0 & 1 & S_1 & 7 & S_2 & S_3 & S_4 & 6 & S_5 & 2 & S_6 & S_7 & 4 \\
\\
  ( &  (  & 1 &  +  & 7 &  )  &  *  & 6 &  -  & 2 &  )  &  /  & 4 \\
S_0 & S_1 & 1 & S_2 & 7 & S_3 & S_4 & 6 & S_5 & 2 & S_6 & S_7 & 4
\end{matrix}
$

We see that we can potentially have 3 symbols between two digits like )*( in the first example, the symbols $S_2 S_3 S_4$ between the digits 7 and 6. 

We could look for expression with 3 symbols before or after each digits, leading to a space search of size $7^{18}$ or about 1.6 quadrillions... And we're not even sure to have covered all cases!

In [5]:
size = 7**18
print(size, ' ~10^', len(str(size))-1, sep='')

1628413597910449 ~10^15


## 4. RPN and binary trees to the rescue

There is another way to represent expression like (1+7)*(6-2)+4 using the [Reverse Polnish Notation (RPN)](https://en.wikipedia.org/wiki/Reverse_Polish_notation) or postfix expression: <br/>
1 7 + 6 2 - * 4 +

RPN evaluates left-to-right using a stack. When a value is seen, it is pushed onto the stack, when an operation is seen, the two last values are poped out, the operation performed, and the result pushed onto the stack.

$
\begin{matrix}
 1 & 7 & + & 6 & 2 & - & *  & 4 & + & expression\\
 \\
 1 & 7 & 8 & 6 & 2 & 4 & 32 & 4 & 36 & stack \\
   & 1 &   & 8 & 6 & 8 &    & 32 \\
   &   &   &   & 8 & 
\end{matrix}
$

Generating the expressions using RPN seems more manageable. However implementing the stack-based evaluation sounds like it will be slow... 

Some 30 years ago (hmm, time flies), I was fascinated by symbolic computations. In a flashback, I remembered the role played by [binary trees](https://en.wikipedia.org/wiki/Binary_expression_tree) to represent symbolic algebraic expressions. In fact, it is not difficult to convert any postfix expression to a binary tree.

```
        +
       / \ 
      /   \
     *     \
    /  \    \
   /    \    \
  +      -    \
/   \   / \    \
1   7   6  2    4
```
The corresponding expression can be built using a [Depth First Search (DFS)](https://en.wikipedia.org/wiki/Depth-first_search). Generally speaking, a tree with an operation $\top$ and 2 sub-trees $L$ and $R$ form the expression $(L)\top(R)$ :
```
  T
 / \
L   R
```
So the tree above leads to the expression $((1+7)*(6-2))+(4)$. 

Mathematically, some braces are superflous. We could get rid of those, but they don't do any harm. And Lisp programmers will love it!

Using binary trees to carve the expressions takes automatically care of the braces combinations. Therefore our initial problem becomes:

> Find all binary trees with 5 leaves representing the digits 1,7,6,2,4 respectively. The 4 nodes correspond to one of the symbol $\{+,-,*,/,\circ \}$. For each tree, evaluate all possible expressions as we did in section [2. First idea](#2.-First-idea-(not-good-enough))

## 5. Generating all the binary trees with 5 leaves

A binary tree has a left $L$ and a right $R$ subtree. If our tree has a total of $n$ leaves, and the left subtree $k$ leaves, then the right subtree has $n-k$ leaves.

```
  T
 / \
L   R
```

We can therefore construct all trees with $n$ leaves, if we know all the trees with $1,...,n-1$ leaves.

```
1-leaf trees
  (.)                     

---------------------------------------
2-leaves trees
   T           
  / \
(.) (.)  (.)T(.)    

---------------------------------------
3-leaves trees
     T
    / \
   /   T
  /   / \
(.) (.) (.)  (.)T((.)T(.))

     T
    / \
   T   \
  / \   \
(.) (.) (.)  ((.)T(.))T(.)

```

In the string representation using the braces, dot and T character, "(.)" corresponds to a leaf (one of the digits 1,7,6...) whereas T to one of the operation/symbol +,-,...

We can almost directly translated this idea into Python code. The function `gentrees` generates all the trees with a given number of leaves. We first start with a forest of 0- and 1-leaf tree `[{}, {'.'}]` and we successively built the trees with 2-, 3-,... leaves. It's handy to represent the trees with k-leaves as a set. `forest`is the list of all tress with 0,1,2,3,...-leaves, forest[k] represents the set of all trees with k-leaves.

The  `gentrees()` function constructs this forest. This is a nice example of [dynamic programming](https://skerritt.blog/dynamic-programming/). The function takes as argument the number of wanted leaves, and an optional `op` argument for the operator symbol (default is `T`). It returns the last item of the forest's list, which is the set of all trees with `n_leaves`.

In [6]:
def gentrees(n_leaves, op='T'):
    # 0 and 1-leaf tree
    forest = [ {}, {'.'} ]

    # generate the forest with trees having 2, 3,...n leaves
    for n in range(2,n_leaves+1):
        trees = set()
        # assume the left subtree has k leaves with k=1,...n-1
        for k in range(1,n):
            for left_tree in forest[k]:
                # then the right subtree must have n-k leaves
                for right_tree in forest[n-k]:
                    trees.add(f"({left_tree}){op}({right_tree})")
        forest.append(trees)

    # the last item corresponds to all the threes with n leaves
    return forest[-1]
    
gentrees(3)

{'((.)T(.))T(.)', '(.)T((.)T(.))'}

## 6. From binary trees to expressions 

The next step is to convert the binary trees obtained with `gentrees()` to an expression we can evaluate with `eval`.  We need to:
1) replace the leaves with the digits 1,7,6,2,4
1) replace the 4 nodes 'T' with one of the possible symbols combinations.

The leaves substitution can be achieved quite easily using a regular expression and `re.sub()`.
For the node substitution, we'll using printf-string representation, and the % string interpolation operator. Let see an example below:

In [7]:
import re

digits=('1','7','6','2','4')
t  = '((.)%s(.))%s(.)' 

# substitute leaves
it = iter(digits)
expr  = re.sub(r'\(\.\)', lambda m: next(it), t)
print(f'{expr = }')

# interpolate the nodes with S
S=('+','-')
print(f'{expr%S = }')

expr = '(1%s7)%s6'
expr%S = '(1+7)-6'


In the example above, we start with the 3-leaves tree `((.)%s(.))%s(.)`. Here the symbol `T` got replaced by `%s`, something we can easily get by passing `op='%s'` to `gentrees`.

We use regular expression to match the string `(.)`, and each match is replaced by the next value in the digits sequence. That is 1st match is replaced by `1`, 2nd match by `7`, 3rd match by `6` etc. After this substitution, we get the string `expr='(1%s7)%s6'`.

We can then replace the two "%s" using `expr%S` where S is a tuple of 2 strings corresponding to the operation we'd like to perform, here in the example `S=('+','-')`.

Let's built upon `gentrees()` a function `genexprs()` that will create a set of trees using '%s' for the nodes, and where the leaves are replaced by the digits 1,7,6,... 

In [8]:
import re

def genexprs(digits):
    n_leaves = len(digits)
    trees = gentrees(n_leaves,'%s')

    result = set()
    for t in trees:
        it = iter(digits)
        s = re.sub(r'\(\.\)', lambda m: next(it), t)
        result.add(s)
    return result

In [9]:
digits=('1','7','6','2','4')
exprs = genexprs(digits)
print("#exprs = ", len(exprs))
exprs

#exprs =  14


{'(((1%s7)%s6)%s2)%s4',
 '((1%s(7%s6))%s2)%s4',
 '((1%s7)%s(6%s2))%s4',
 '((1%s7)%s6)%s(2%s4)',
 '(1%s((7%s6)%s2))%s4',
 '(1%s(7%s(6%s2)))%s4',
 '(1%s(7%s6))%s(2%s4)',
 '(1%s7)%s((6%s2)%s4)',
 '(1%s7)%s(6%s(2%s4))',
 '1%s(((7%s6)%s2)%s4)',
 '1%s((7%s(6%s2))%s4)',
 '1%s((7%s6)%s(2%s4))',
 '1%s(7%s((6%s2)%s4))',
 '1%s(7%s(6%s(2%s4)))'}

## 7. Answering the puzzle

For each expressions, we can then test all symbols combination as we did in section [2. First idea](#2.-First-idea-(not-good-enough)). As we have only 14 kind of expressions, the size of the search space is $14*5^4 = 8750$, which is easily explored. 

In [10]:
from itertools import product

symbols = ('+','-','*','/','')
digits = ('1','7','6','2','4')

for e in genexprs(digits):
    it=product(symbols, repeat=4)
    for n,s in enumerate(it):
        expr = e%s
        print('checking: ',expr)
        if eval(expr)==100:
            print(expr)

checking:  1+((7+(6+2))+4)
checking:  1+((7+(6+2))-4)
checking:  1+((7+(6+2))*4)
checking:  1+((7+(6+2))/4)
checking:  1+((7+(6+2))4)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<string>, line 1)

Sounds promizing, but certain expressions like $(1+(7+(6+2)))4 \quad$ aren't a valid Python expression. So we need to handle the corresponding exception, here SyntaxError. There are a few more corner cases to handle, like:

$(1+(7(6+2)))+4$  raises TypeError<br/>
$(1+7)/((6-2)-4)$ raises ZeroDivisionError

In addition, jupyter will trigger a syntax warning, that we suppress to avoid swamping the output. 

In [11]:
# ignore warning messages from jupyter
import warnings
warnings.filterwarnings('ignore')

In [12]:
from itertools import product

digits=('1','7','6','2','4')
symbols=('+','-','*','/','')

for e in genexprs(digits):
    it=product(symbols, repeat=4)
    for ops in it:
        expr = e%ops
        try:
            v = eval(expr)
            if v==100:
                print(expr)
        except (SyntaxError, TypeError, ZeroDivisionError):
            pass

(1*(76))+(24)
((17)*6)+(2-4)
1*((76)+(24))
((1+7)*(6*2))+4
((17)+(6+2))*4
(((1+7)*6)*2)+4
(((17)+6)+2)*4
(((17)*6)+2)-4


## 8. Aftermath (ah,ah)

The solutions output contain some superfluous braces. While we could teach Python how to remove those braces, a human can do the job quicker:

$
\begin{flalign*}
& 1*76+24     \\
& 1*(76+24)   \\
& (17+6+2)*4  \\
& 17*6+2-4    \\
& (1+7)*6*2+4
\end{flalign*}
$

We missed the two first solutions. 

Finding a solution by hand was fun, finding all solutions with Python even more so. This simple looking puzzle took us to the realm of iterators and itertools, binary trees, dynamic programming, regular expression, lambda, string interpolation... What a journey!

I kept the code 'relatively' straightforward to ease the understanding. "More Pythonic / efficient" versions exist, but it's good enough for now.

Happy to hear if you find any mistakes or have - possibly more elegants -approaches to solve the problem.

# HAPPY END 😎 